In [1]:
import json

test_data = []

with open('text-to-code/dataset/concode/test.json') as f:
  for line in f:
    test_data.append(json.loads(line))

len(test_data)

2000

In [2]:
for i in range(len(test_data)):
  test_data[i]['input'] = test_data[i]['nl'] + '<|sepoftext|>'

print(test_data[0])

{'code': 'void function ( ScriptOrFnNode arg0 ) { int loc0 = - 1 ; collectFuncNodes ( arg0 , loc0 , null ) ; }', 'nl': 'generate mappings for each function node and parameters and variables names associated with it . concode_field_sep int parentScope concode_elem_sep ArrayList functionBracePositions concode_elem_sep ObjArray funcObjects concode_elem_sep int functionNum concode_elem_sep ArrayList functionVarMappings concode_elem_sep int lastTokenCount concode_elem_sep ArrayList replacedTokens concode_field_sep boolean isInScopeChain concode_elem_sep void reset concode_elem_sep void leaveNestingLevel concode_elem_sep String getMappedToken concode_elem_sep String getPreviousTokenMapping concode_elem_sep void collectFuncNodes concode_elem_sep int sourceCompress concode_elem_sep void enterNestingLevel', 'input': 'generate mappings for each function node and parameters and variables names associated with it . concode_field_sep int parentScope concode_elem_sep ArrayList functionBracePositions

In [3]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained("distilgpt2")
model = GPT2LMHeadModel.from_pretrained("results/checkpoint-73500")

tokenizer.add_special_tokens({'sep_token': '<|sepoftext|>'})

tokenizer.pad_token = tokenizer.eos_token

/home/cslee/anaconda3/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [4]:
encoded_input = tokenizer(test_data[0]['input'], return_tensors='pt')

print(len(encoded_input['input_ids'][0]))
print(test_data[0]['input'])

gen_tokens = model.generate(encoded_input['input_ids'], 
                            max_length=model.config.n_positions,
                            pad_token_id=tokenizer.eos_token_id, 
                            num_return_sequences=1)

gen_text = tokenizer.batch_decode(gen_tokens, skip_special_tokens=False)[0]

sep_index = gen_text.find('<|sepoftext|>')

print(gen_text[sep_index+len(tokenizer.sep_token):-len(tokenizer.eos_token)])

206
generate mappings for each function node and parameters and variables names associated with it . concode_field_sep int parentScope concode_elem_sep ArrayList functionBracePositions concode_elem_sep ObjArray funcObjects concode_elem_sep int functionNum concode_elem_sep ArrayList functionVarMappings concode_elem_sep int lastTokenCount concode_elem_sep ArrayList replacedTokens concode_field_sep boolean isInScopeChain concode_elem_sep void reset concode_elem_sep void leaveNestingLevel concode_elem_sep String getMappedToken concode_elem_sep String getPreviousTokenMapping concode_elem_sep void collectFuncNodes concode_elem_sep int sourceCompress concode_elem_sep void enterNestingLevel<|sepoftext|>
int function ( ) { return funcObjects. size ( ) ; }


In [5]:
#encoded_inputs = [tokenizer.encode(d['input'], return_tensors='pt', truncation=True, max_length=model.config.n_positions) for d in test_data]
encoded_inputs = [tokenizer.encode(d['input'], return_tensors='pt', truncation=True, max_length=model.config.n_positions - 64) for d in test_data]

In [6]:
import tqdm

gen_ans = []

for i in tqdm.tqdm(range(len(encoded_inputs))):
  #if len(encoded_inputs[i][0]) > model.config.n_positions // 100 * 100:
  #  gen_ans.append(torch.tensor([]))
  #else:
  gen_ans.append(model.generate(encoded_inputs[i], 
                                max_length=model.config.n_positions,
                                pad_token_id=tokenizer.eos_token_id, 
                                num_return_sequences=1)[0])



100%|██████████| 2000/2000 [45:52<00:00,  1.38s/it]  


In [7]:
gen_text = tokenizer.batch_decode(gen_ans, skip_special_tokens=False)
gen_code = []

for i in gen_text:
  sep_index = i.find('<|sepoftext|>')
  if sep_index == -1:
    gen_code.append('')
    continue
  
  eos_index = i.find(tokenizer.eos_token)
  if eos_index == -1:
    gen_code.append(i[sep_index+len(tokenizer.sep_token):])
  else:
    gen_code.append(i[sep_index+len(tokenizer.sep_token):eos_index])

print(gen_code[0])

int function ( ) { return funcObjects. size ( ) ; }


In [8]:
import copy

test_data_processed = copy.deepcopy(test_data)

for i in range(len(gen_code)):
  if gen_code[i] == '':
    test_data_processed[i]['code'] = ''

In [9]:
# eval dataset

dev_data = []

with open('text-to-code/dataset/concode/dev.json') as f:
  for line in f:
    dev_data.append(json.loads(line))

for i in range(len(dev_data)):
  dev_data[i]['input'] = dev_data[i]['nl'] + '<|sepoftext|>'

In [10]:
# eval dataset

dev_data = []

with open('text-to-code/dataset/concode/dev.json') as f:
  for line in f:
    dev_data.append(json.loads(line))

for i in range(len(dev_data)):
  dev_data[i]['input'] = dev_data[i]['nl'] + '<|sepoftext|>'

encoded_dev_inputs = [tokenizer.encode(d['input'], return_tensors='pt', truncation=True, max_length=model.config.n_positions) for d in dev_data]


gen_dev_ans = []

for i in tqdm.tqdm(range(len(encoded_dev_inputs))):
  gen_dev_ans.append(model.generate(encoded_dev_inputs[i], 
                                    max_length=model.config.n_positions,
                                    pad_token_id=tokenizer.eos_token_id, 
                                    num_return_sequences=1)[0])

 18%|█▊        | 364/2000 [07:01<18:31,  1.47it/s]  /home/cslee/anaconda3/lib/python3.9/site-packages/transformers/generation/utils.py:1268: UserWarning: Input length of input_ids is 1024, but `max_length` is set to 1024. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(
100%|██████████| 2000/2000 [41:30<00:00,  1.25s/it]  


In [11]:
gen_dev_text = tokenizer.batch_decode(gen_dev_ans, skip_special_tokens=False)
gen_dev_code = []

for i in gen_dev_text:
  sep_index = i.find('<|sepoftext|>')
  if sep_index == -1:
    gen_dev_code.append('')
    continue

  eos_index = i.find(tokenizer.eos_token)
  if eos_index == -1:
    gen_dev_code.append(i[sep_index+len(tokenizer.sep_token):])
  else:
    gen_dev_code.append(i[sep_index+len(tokenizer.sep_token):eos_index])

dev_data_processed = copy.deepcopy(dev_data)

for i in range(len(gen_dev_code)):
  if gen_dev_code[i] == '':
    dev_data_processed[i]['code'] = ''

In [12]:
import numpy as np
import sys
import nbimporter
from bleu import _bleu
import os 

def cal_bleu(gen_code, test_data):
  
  total = len(gen_code)
  EM = 0.0

  # 为了和原 repo 中的 _bleu 函数输入保持一致
  with open("ground_truth.txt", "w") as wf:
    for pred_code, labels_code in zip(gen_code, [d['code'] for d in test_data]):  
        pred_code = pred_code.strip()
        labels_code = labels_code.strip()
        wf.write(labels_code+"\n")
        
        if pred_code == labels_code:
           EM += 1 

  with open("preds.txt", "w") as wf:
      for pred_code in gen_code:
         pred_code = pred_code.strip()
         wf.write(pred_code+"\n")

        
  bleu_score = round(_bleu("ground_truth.txt", "preds.txt"), 2)
        
  print(f"BLEU: {bleu_score}, EM: {round(EM/total*100, 2)}")

  try:
      os.remove("preds.txt")
      os.remove("ground_truth.txt")
  except Exception:
      pass
  
cal_bleu(gen_code, test_data)
cal_bleu(gen_code, test_data_processed)

BLEU: 19.68, EM: 8.4
BLEU: 19.81, EM: 9.55


In [13]:
cal_bleu(gen_dev_code, dev_data)
cal_bleu(gen_dev_code, dev_data_processed)

BLEU: 18.54, EM: 8.45
BLEU: 18.6, EM: 9.05
